In [1]:
!nvidia-smi

Sat Dec 30 08:52:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000001:00:00.0 Off |                    0 |
| N/A   32C    P0    70W / 400W |      0MiB / 81920MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-SXM...  On   | 00000002:00:00.0 Off |                    0 |
| N/A   

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import json
from peft import PeftModel

TORCH_DTYPE = 'bfloat16'
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=getattr(torch, TORCH_DTYPE)
)

In [3]:
from transformers.trainer_utils import get_last_checkpoint

latest = get_last_checkpoint("mixtral-deepspeed")
latest

'mixtral-deepspeed/checkpoint-2320'

In [4]:
tokenizer = AutoTokenizer.from_pretrained(latest)

In [5]:
base_model = AutoModelForCausalLM.from_pretrained(
    'mistralai/Mixtral-8x7B-Instruct-v0.1',
    use_flash_attention_2 = True,
    quantization_config = nf4_config
)

The model was loaded with use_flash_attention_2=True, which is deprecated and may be removed in a future release. Please use `attn_implementation="flash_attention_2"` instead.


Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

In [6]:
model = PeftModel.from_pretrained(base_model, latest)
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MixtralForCausalLM(
      (model): MixtralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MixtralDecoderLayer(
            (self_attn): MixtralFlashAttention2(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer)

In [7]:
def parse_mistral_chat(messages, function_call = None):

    user_query = messages[-1]['content']

    users, assistants = [], []
    for q in messages[:-1]:
        if q['role'] == 'user':
            users.append(q['content'])
        elif q['role'] == 'assistant':
            assistants.append(q['content'])

    texts = ['<s>']
    
    if function_call:
        fs = []
        for f in function_call:
            f = json.dumps(f, indent=4)
            fs.append(f)
        fs = '\n\n'.join(fs)
        texts.append(f'\n[FUNCTIONCALL]\n{fs}\n')
        
    for u, a in zip(users, assistants):
        texts.append(f'[INST] {u.strip()} [/INST] {a.strip()}</s>')

    texts.append(f'[INST] {user_query.strip()} [/INST]')
    prompt = ''.join(texts).strip()
    return prompt

In [9]:
s = """
camne nak code sentiment analysis guna python3
"""

messages = [
    {'role': 'user', 'content': s.strip()},
]
prompt = parse_mistral_chat(messages)
inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=128,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = model.generate(**generate_kwargs)
print(tokenizer.decode(r[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] camne nak code sentiment analysis guna python3 [/INST] Untuk mengimplementasikan sentiment analysis menggunakan Python 3, anda boleh menggunakan beberapa pustaka yang tersedia seperti:

1. `TextBlob`: Pustaka yang boleh digunakam untuk melakukan pelbagai analisis teks seperti analisis kesan semula jadi, pengenalan entiti, pengenalan jenis, dan lain-lain. Untuk melakukan sentiment analysis, and


In [13]:
s = """
terjemah `tak suka awak` ke inggeris
"""

messages = [
    {'role': 'user', 'content': s.strip()},
]
prompt = parse_mistral_chat(messages)
inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=128,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = model.generate(**generate_kwargs)
print(tokenizer.decode(r[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] terjemah `tak suka awak` ke inggeris [/INST] I don't like you.</s>
